# Lab Three: Extending Logistic Regression
 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Preparation and Overview

#### 1.1 Business Case

Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results. For example, would the model be deployed or used mostly for offline analysis? 

#### 1.2 Preparation of Data

In [103]:
import pandas as pd
import numpy as np

# Define and prepare your class variables.
df = pd.read_csv('wine_dataset/winequalityN.csv')
df = df[df['type']=='white']
df = df.drop(columns = ['type'])
df = df.dropna()
X = df.drop(columns = ['quality'])
y = df['quality']
# Use proper variable representations (int, float, one-hot, etc.).
# Use pre-processing methods (as needed) for dimensionality reduction, 
# scaling, etc. Remove variables that are not needed/useful for the analysis. 
# Describe the final dataset that is used for classification/regression
display(X.info())
display(y.info())
# (include a description of any newly formed variables you created).
# MAKE SURE TO NORMALIZE VALUES

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4897
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4870 non-null   float64
 1   volatile acidity      4870 non-null   float64
 2   citric acid           4870 non-null   float64
 3   residual sugar        4870 non-null   float64
 4   chlorides             4870 non-null   float64
 5   free sulfur dioxide   4870 non-null   float64
 6   total sulfur dioxide  4870 non-null   float64
 7   density               4870 non-null   float64
 8   pH                    4870 non-null   float64
 9   sulphates             4870 non-null   float64
 10  alcohol               4870 non-null   float64
dtypes: float64(11)
memory usage: 456.6 KB


None

<class 'pandas.core.series.Series'>
Int64Index: 4870 entries, 0 to 4897
Series name: quality
Non-Null Count  Dtype
--------------  -----
4870 non-null   int64
dtypes: int64(1)
memory usage: 76.1 KB


None

#### 1.3 Division of Trainig and Testing Data

In [104]:
# Divide your data into training and testing data using an 80% training 
# and 20% testing split. Use the cross validation modules that are part 
# of scikit-learn.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?  

In [105]:
### 2. Modeling

#### 2.1 One-Versus-All Logistic Regression Classifier

In [106]:
from scipy.special import expit
from sklearn.metrics import accuracy_score

class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'

    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))

    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1

    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction


    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate
            # add bacause maximizing

# for this, we won't perform our own BFGS implementation
# (it takes a fair amount of code and understanding, which we haven't setup yet)
# luckily for us, scipy has its own BFGS implementation:
from scipy.optimize import fmin_bfgs # maybe the most common bfgs algorithm in the world
from numpy import ma
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):

    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(w**2)
        #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient

    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)

        self.w_ = self.w_.reshape((num_features,1))

class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20,
                 C=0.0001,
                 solver=BFGSBinaryLogisticRegression):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem

            # train the binary classifier for this class

            hblr = self.solver(eta=self.eta,iterations=self.iters,C=self.C)
            hblr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(hblr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier

        return np.hstack(probs) # make into single matrix

    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row

In [107]:
%%time
lr = MultiClassLogisticRegression(eta=1,
                                  iterations=10,
                                  C=0.01,
                                  solver=BFGSBinaryLogisticRegression
                                  )
lr.fit(X,y)
print(lr)
yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))
unique_yhat, counts_yhat = np.unique(yhat, return_counts=True)
unique_y, counts_y = np.unique(y, return_counts=True)
print(np.asarray((unique_yhat, counts_yhat)).T)
print(np.asarray((unique_y, counts_y)).T)

MultiClass Logistic Regression Object with coefficients:
[[-0.04930964 -0.13381531 -0.00209226 -0.01006834 -0.0838108  -0.00059989
   0.01787291 -0.00294707 -0.03349794 -0.10861926 -0.01787619 -0.37767188]
 [-0.00922379  0.06277083  0.03911189 -0.0115826  -0.04957062  0.00149078
  -0.06508956  0.00394881 -0.0045812  -0.01618247 -0.0070719  -0.20846836]
 [ 0.06986028  0.34329671  0.09536574  0.00733588 -0.01273653  0.01468912
  -0.01208132  0.0097361   0.06640493  0.16743208 -0.01753883 -0.46702099]
 [-0.01932771 -0.08663188 -0.13958461  0.03052666  0.01402905 -0.0031808
   0.00509632 -0.00225901 -0.01763659 -0.05625123  0.00935413  0.06428658]
 [-0.09413913 -0.41986038 -0.05756825 -0.02153922 -0.00539694 -0.01730121
   0.0119446  -0.01046589 -0.08462634 -0.22946473  0.00769617  0.3044183 ]
 [-0.03834349 -0.31781933 -0.01097037 -0.01139994  0.00327578 -0.00449756
   0.02743831 -0.0189195  -0.03791695 -0.10079116 -0.01597708  0.06660213]
 [-0.06180066 -0.22031426 -0.01468611 -0.00531877 

In [108]:
%%time
from sklearn.linear_model import LogisticRegression as SKLogisticRegression

lr_sk = SKLogisticRegression(solver='liblinear') # all params default

lr_sk.fit(X,y)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

[[-6.69492099e-01  5.40250299e-01  7.96203917e-01 -4.74192393e-01
  -8.81037466e-02  5.07314375e-02  2.58806856e-02  1.51678776e-03
  -6.63247741e-01 -1.29113450e+00 -5.84990819e-01 -4.25845876e-01]
 [-7.33792911e-02  2.92105683e-01  4.39657377e+00 -6.06222872e-01
  -8.26519489e-02  1.03392357e-01 -4.79793403e-02 -1.39052006e-03
  -4.61493085e-02  4.54103967e-01 -1.16343142e-01 -5.58478324e-01]
 [ 3.01029025e+00  1.14120058e-01  3.61362275e+00  2.31290486e-01
  -4.41178025e-02  5.41961314e-01 -8.48962106e-03  4.28860584e-03
   2.99587148e+00  2.64637655e-01 -1.20601146e+00 -8.78290529e-01]
 [-7.59539105e-03 -6.19469730e-02 -3.35565623e+00  1.72983062e-01
   2.28163648e-02  2.79529409e-01  7.21265670e-04 -1.41656905e-04
   1.18852463e-02 -1.95127236e-01  1.61095011e-01  1.38691202e-01]
 [-3.52750737e+00 -1.05773142e-01 -3.01386648e+00 -7.04012803e-01
   2.81016566e-02 -1.44024018e+00  8.81147025e-03 -3.99916153e-03
  -3.53583853e+00 -1.88214372e-01  1.12173914e+00  6.89559439e-01]
 [-2.

#### 2. Training Classifier for Good Generalization Performance

Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

#### 2.3 Comparing Best Performing Procedure to Scikit-Learn

In [109]:
# Visualize the performance differences in terms of training time and classification performance.

Discuss the results. 

### 3. Deployment

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

### 4. BFGS (Can change but thought this would be better)

In [110]:
# Implementation of BFGS

Compare your performance accuracy and runtime to the BFGS implementation in SciPy (that we used in lecture). 
